In [ ]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
np.random.seed(666)

physical_devices = tf.config.list_physical_devices('GPU')

tf.config.experimental.set_memory_growth(physical_devices[0], True)

In [ ]:
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

# Séparation des images en ensembles d'entraînement, validation et test

In [ ]:
import os, shutil 

original_dataset_dir= 'D:/Documents/SYS866_Projet_Final/Grad-CAM/HCP_SYS866_2020/'

base_dir='D:/Documents/SYS866_Projet_Final/Grad-CAM/cerveaux/'
if not os.path.exists(base_dir):
    os.mkdir(base_dir)
    
train_dir=os.path.join(base_dir,'train/')
if not os.path.exists(train_dir):
    os.mkdir(train_dir)
    
validation_dir=os.path.join(base_dir,'validation/')
if not os.path.exists(validation_dir):
    os.mkdir(validation_dir)
    
test_dir=os.path.join(base_dir,'test/')
if not os.path.exists(test_dir):
    os.mkdir(test_dir)

In [ ]:
train_homme_dir=os.path.join(train_dir,'homme')
if not os.path.exists(train_homme_dir):
    os.mkdir(train_homme_dir)

train_femme_dir = os.path.join(train_dir, 'femme')
if not os.path.exists(train_femme_dir):
    os.mkdir(train_femme_dir)
    
validation_homme_dir=os.path.join(validation_dir,'homme')
if not os.path.exists(validation_homme_dir):
    os.mkdir(validation_homme_dir)

validation_femme_dir = os.path.join(validation_dir, 'femme')
if not os.path.exists(validation_femme_dir):
    os.mkdir(validation_femme_dir)
    
test_homme_dir=os.path.join(test_dir,'homme')
if not os.path.exists(test_homme_dir):
    os.mkdir(test_homme_dir)

test_femme_dir = os.path.join(test_dir, 'femme')
if not os.path.exists(test_femme_dir):
    os.mkdir(test_femme_dir)

In [ ]:
file= open('D:/Documents/SYS866_Projet_Final/Grad-CAM/labels.csv','r')

ligne= file.readlines()

file.close()

In [ ]:
nom=[]
sexe=[]


for i in range (2,len(ligne)):
    
    li=str(ligne[i]).rstrip('\n')
    liste=li.split(',')
    nom.append(liste[0])
    sexe.append(liste[8])
    
print(len(nom))
print(len(sexe))

In [ ]:
# Ajout d'images au dossier d'entrainement
for i in range(0, 1000):
    fname='img'+nom[i]+'_T1w.png'
    src= os.path.join(original_dataset_dir, fname)
    if os.path.isfile(src)==True:
        if sexe[i]=='1':
            src= os.path.join(original_dataset_dir, fname)
            dst=os.path.join(train_homme_dir, fname)
            shutil.copyfile(src, dst)
        else:
            src= os.path.join(original_dataset_dir, fname)
            dst=os.path.join(train_femme_dir, fname)
            shutil.copyfile(src, dst)

In [ ]:
# Ajout d'images au dossier de validation
for i in range(1000, 1100):
    fname='img'+nom[i]+'_T1w.png'
    src= os.path.join(original_dataset_dir, fname)
    if os.path.isfile(src)==True:
        if sexe[i]=='1':
            src= os.path.join(original_dataset_dir, fname)
            dst=os.path.join(validation_homme_dir, fname)
            shutil.copyfile(src, dst)
        else:
            src= os.path.join(original_dataset_dir, fname)
            dst=os.path.join(validation_femme_dir, fname)
            shutil.copyfile(src, dst)

In [ ]:
# Ajout d'images au dossier de test
for i in range(1100, 1205):
    fname='img'+nom[i]+'_T1w.png'
    src= os.path.join(original_dataset_dir, fname)
    if os.path.isfile(src)==True:
        if sexe[i]=='1':
            src= os.path.join(original_dataset_dir, fname)
            dst=os.path.join(test_homme_dir, fname)
            shutil.copyfile(src, dst)
        else:
            src= os.path.join(original_dataset_dir, fname)
            dst=os.path.join(test_femme_dir, fname)
            shutil.copyfile(src, dst)

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.10,
        brightness_range=[0.5,1.5],
        #horizontal_flip=True,
        rotation_range=5)

validation_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

In [ ]:
image_size = (150, 150)
batch_size = 32

training_set = train_datagen.flow_from_directory(
    "cerveaux/train",
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical'
)

In [ ]:
validation_set = validation_datagen.flow_from_directory(
    "cerveaux/validation",
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical'
)

In [ ]:
test_datagen = ImageDataGenerator(rescale=1./255)

test_set = test_datagen.flow_from_directory(
    "cerveaux/test",
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False
)

## Définition du modèle et entraînement

In [ ]:
from tensorflow.keras import layers
from tensorflow.keras import models

model = models.Sequential()
model.add(layers.Conv2D(32, (3,3),activation='relu', input_shape=(150,150,3)))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Conv2D(64, (3,3), activation ='relu'))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Conv2D(128, (3,3), activation ='relu'))
model.add(layers.MaxPooling2D((2,2)))

model.add(layers.Flatten())
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(2, activation='softmax'))

In [ ]:
model.summary()

In [ ]:
epochs = 30

model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=['accuracy'])

history = model.fit(x = training_set, validation_data=validation_set, shuffle=True, epochs=epochs)

## Traçage des graphiques d'entraînement et ROC

In [ ]:
import matplotlib.pyplot as plt

acc= history.history['accuracy']
val_acc = history.history['val_accuracy']
loss= history.history['loss']
val_loss= history.history['val_loss']

epochs= range(1, len(acc)+1)

plt.plot(epochs, acc, 'bo', label='entrainement')
plt.plot(epochs, val_acc,'b', label='validation')
plt.title('Exactitude pendant l\'entrainement et la validation')
plt.legend()

plt.figure()
plt.plot(epochs, loss, 'bo', label='entrainement')
plt.plot(epochs, val_loss, 'b', label='validation')
plt.title('Perte pendant l\'entrainement et la validation')
plt.legend()
plt.show()

In [ ]:
test_loss, test_tp, test_fp, test_tn, test_fn, test_acc, test_prec, test_recall, test_auc = model.evaluate(test_set)

In [ ]:
print(test_loss, test_tp, test_fp, test_tn, test_fn, test_acc, test_prec, test_recall, test_auc)

In [ ]:
from keras.preprocessing import image

test_image = image.load_img('./cerveaux/test/homme/img999517_T1w.png', target_size = (150, 150))
test_image = image.img_to_array(test_image)/255.
test_image = np.expand_dims(test_image, axis=0)

prediction = model.predict_on_batch(test_image)
print(prediction)

In [ ]:
from keras.preprocessing import image

def test_image(IMAGE_PATH):
    test_image = image.load_img(IMAGE_PATH, target_size = (150, 150))
    test_image = image.img_to_array(test_image)/255.
    test_image = np.expand_dims(test_image, axis=0)

    prediction = model.predict_on_batch(test_image)
    
    if prediction[0,0] < prediction[0,1]:
        print(IMAGE_PATH.split("/")[-1])
        print(prediction)

In [ ]:
import os

for filename in os.listdir('./cerveaux/test/femme/'):
    
    test_image('./cerveaux/test/femme/' + filename)

In [ ]:
y_true_labels = test_set.classes
true_labels = []

for label in y_true_labels:
    true_labels.append(np.array([np.float32(label)]))

In [ ]:
preds = model.predict(test_set)

predictions = []

for x in preds.tolist():
    predictions.append(x[1])

In [ ]:
from sklearn.metrics import roc_curve, auc

In [ ]:
fpr, tpr, _ = roc_curve(true_labels, predictions)
auc = auc(fpr, tpr)

In [ ]:
plt.figure(1)
plt.plot([0, 1], [0, 1], 'k--')
plt.plot(fpr, tpr, label='AUC = {:.3f}'.format(auc))
plt.xlabel('False positive rate')
plt.ylabel('True positive rate')
plt.title('ROC curve')
plt.legend(loc='best')
plt.show()

# Sauvegarde du modèle

In [ ]:
model.save('D:/Documents/SYS866_Projet_Final/Grad-CAM/models/4_couches_adam_97_62_categorical')

### Pour juste loader le modèle

In [ ]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt

model = keras.models.load_model('D:/Documents/SYS866_Projet_Final/Grad-CAM/models/4_couches_adam_97_62_categorical')

# Grad-CAM

#### https://gist.github.com/RaphaelMeudec/e9a805fa82880876f8d89766f0690b54

In [ ]:
import cv2
import numpy as np
import tensorflow as tf
from tensorflow import keras

#physical_devices = tf.config.list_physical_devices('GPU')

#tf.config.experimental.set_memory_growth(physical_devices[0], True)

In [ ]:
def grad_cam(CLASS_INDEX, LAYER_NAME, IMAGE_PATH, MODEL_PATH):
    img = tf.keras.preprocessing.image.load_img(IMAGE_PATH, target_size=(150, 150))
    img = tf.keras.preprocessing.image.img_to_array(img)/255.

    model = keras.models.load_model(MODEL_PATH)
    
    grad_model = tf.keras.models.Model([model.inputs], [model.get_layer(LAYER_NAME).output, model.output])

    with tf.GradientTape() as tape:
        conv_outputs, predictions = grad_model(np.array([img]))
        loss = predictions[:, CLASS_INDEX]

    output = conv_outputs[0]
    grads = tape.gradient(loss, conv_outputs)[0]

    gate_f = tf.cast(output > 0, 'float32')
    gate_r = tf.cast(grads > 0, 'float32')
    guided_grads = tf.cast(output > 0, 'float32') * tf.cast(grads > 0, 'float32') * grads

    weights = tf.reduce_mean(guided_grads, axis=(0, 1))

    cam = np.ones(output.shape[0: 2], dtype = np.float32)

    for i, w in enumerate(weights):
        cam += w * output[:, :, i]

    cam = cv2.resize(cam.numpy(), (150, 150))
    cam = np.maximum(cam, 0)
    heatmap = (cam - cam.min()) / (cam.max() - cam.min())

    cam = cv2.applyColorMap(np.uint8(255*heatmap), cv2.COLORMAP_JET)

    output_image = cv2.addWeighted(cv2.cvtColor(img.astype('uint8'), cv2.COLOR_RGB2BGR), 0.5, cam, 1, 0)

    
    name = IMAGE_PATH.split("/")[-1].split(".")[0]
    true_class = IMAGE_PATH.split("/")[-2]
    cv2.imwrite('./pictures/' + name + '_' + str(CLASS_INDEX) + '_' + true_class + '.png', output_image)

In [ ]:
HOMME_CLASS_INDEX = 1
FEMME_CLASS_INDEX = 0

MODEL_PATH = "D:/Documents/SYS866_Projet_Final/Grad-CAM/models/4_couches_adam_97_62_categorical"
image_paths = ['./cerveaux/test/femme/img915604_T1w.png',
               './cerveaux/test/femme/img920479_T1w.png',
               './cerveaux/test/femme/img925619_T1w.png',
               './cerveaux/test/femme/img927528_T1w.png',
               './cerveaux/test/femme/img975176_T1w.png',
               './cerveaux/test/femme/img948145_T1w.png',
               './cerveaux/test/femme/img915604_T1w_flipped.png',
               './cerveaux/test/femme/img920479_T1w_flipped.png']
CLASS_INDEX = FEMME_CLASS_INDEX
LAYER_NAME = 'conv2d_2'

In [ ]:
for IMAGE_PATH in image_paths:
    grad_cam(CLASS_INDEX, LAYER_NAME, IMAGE_PATH, MODEL_PATH)

In [ ]:
from PIL import Image, ImageOps 

im1 = Image.open('./cerveaux/test/femme/img915604_T1w.png')
im2 = Image.open('./cerveaux/test/femme/img920479_T1w.png')

im_mirror1 = ImageOps.mirror(im1)
im_mirror2 = ImageOps.mirror(im2)

im_mirror1.save('./cerveaux/test/femme/img915604_T1w_mirror.png', quality=100)
im_mirror2.save('./cerveaux/test/femme/img920479_T1w_mirror.png', quality=100)